In [13]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-3-5-haiku-latest"

In [14]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [15]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 5 objects.
"""
    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    response = chat(messages, stop_sequences=["```"])
    dataset = json.loads(response.strip())
    return dataset

In [16]:
ds = generate_dataset()
ds

[{'task': 'Create a Python function to extract the AWS region from an ARN (Amazon Resource Name)'},
 {'task': 'Write a JSON schema to validate an AWS Lambda function configuration'},
 {'task': 'Develop a regular expression to match valid AWS EC2 instance types (e.g., t2.micro, m5.large)'},
 {'task': 'Create a Python function to convert an AWS CloudFormation template from YAML to JSON'},
 {'task': 'Write a JSON object representing a basic AWS IAM policy allowing S3 read-only access to a specific bucket'}]

In [17]:
with open('test_data.json', 'w') as f:
    json.dump(ds, f, indent=4)